# Exercise 3 (Due: January 25, 2019 before midnight)

The purpose of this exercise is to help you get familiar with MongoDB and the pymongo library. Follow the instructions below to complete the exercise. Use the template notebook below to write and execute your python program. Submit the notebook via D2L. Make sure the notebook also displays the query results. You should also convert the notebook to HTML and submit the HTML file as well.

In this exercise, you need to write a Jupyter notebook script that will open a sample data file named bike_rental.csv data file and store the data in MongoDB. You will also write a few simple queries to retrieve information from the database. The original dataset was obtained from https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset. The sample data contain the total number of bike rentals on each day for a 2-year period from 2011 to 2012. There should be 7 columns (fields) in the sample file: *date, season, holiday, weekday, workingday, weather, and rentalcount*. Make sure you use the sample data from the class website instead of the original raw data.

**Step 1.** Install MongoDB to your machine and launch the MongoDB server on your machine.

**Step 2.** Write a python script to load the data file and store each record in MongoDB using pymongo. To do this, your script should perform the following steps: 

- Open a database connection to the MongoDB server and use the "test" database to store the data.

- Open the input file "bike_rental.csv".

- Read the first line of the input file and extract the names of each field. Store the field names in a list called field_names. Hint: you may use the split() function to parse each line, using comma as the delimiter. See the lecture2.html sample code on "Loading CSV file with Standard Python Library". 

- For each subsequent lines, extract the corresponding fields and store them in a dictionary object named *record*. Note that all the fields must be stored as integer-valued except for date, which is a string. Use pymongo to insert the record into a collection named "bike" in MongoDB.

- After processing all the lines in the input file, count the number of records stored in the database.

- Close the database connection. 

In [1]:
import pymongo

try:     
    connection = pymongo.MongoClient('localhost', 27017)
    db = connection['test']
    
    if db.bike.count_documents({}) > 0:
        db.bike.drop()

    # Open the bike_rental.csv file 
    # Read the first line and extract the field names
    # For each remaining lines, extract the fields and insert them into the database one record at a time

    with open('bike_rental.csv','r') as f:
        field_names = f.readline().strip().split(',')       # extract the field names
        for line in f:
            fields = line.strip().split(',')                # extract list of values for all the fields 
            record = {}                                     # initialize the record
            for i in range(len(fields)):
                if i == 0:
                    record[field_names[i]] = fields[i]      # extract the date string
                else:
                    record[field_names[i]] = int(fields[i]) # extract the remaining fields as integers
            db.bike.insert_one(record)      
            
    numRecords = db.bike.count_documents({})                # count number of records in bike collection
    print("Number of records = ", numRecords)
    connection.close()

except pymongo.errors.ConnectionFailure as e:
    print("Could not connect to MongoDB: %s" % e)

Number of records =  731


**c.** Write a query to display the number of bikes rented on 1/1/2012 

In [2]:
import pymongo

try:
    connection = pymongo.MongoClient('localhost', 27017)
    db = connection['test']

    # Query to find the record for 1/1/2012 

    for result in db.bike.find({"date": "1/1/2012"}):
        count = result['rentalcount']
        print("Number of bikes rented on 1/1/2012 is", count)

    connection.close()
    
except pymongo.errors.ConnectionFailure as e:
    print("Could not connect to MongoDB: %s" % e)

Number of bikes rented on 1/1/2012 is 2294


**d.** Write a query to display the number of bikes rented on each public holiday dates (i.e., when holiday = 1).

In [3]:
import pymongo

try:
    connection = pymongo.MongoClient('localhost', 27017)
    db = connection['test']

    for record in db.bike.find({"holiday":1}):
        rental_date = record['date']
        rental_count = record['rentalcount']
        print('Date =', rental_date, ' Count =', rental_count)
    
    connection.close()

except pymongo.errors.ConnectionFailure as e:
    print("Could not connect to MongoDB: %s" % e)

Date = 1/17/2011  Count = 1000
Date = 2/21/2011  Count = 1107
Date = 4/15/2011  Count = 3126
Date = 5/30/2011  Count = 4098
Date = 7/4/2011  Count = 6043
Date = 9/5/2011  Count = 3351
Date = 10/10/2011  Count = 5117
Date = 11/11/2011  Count = 3368
Date = 11/24/2011  Count = 1495
Date = 12/26/2011  Count = 1317
Date = 1/2/2012  Count = 1951
Date = 1/16/2012  Count = 2298
Date = 2/20/2012  Count = 3129
Date = 4/16/2012  Count = 6370
Date = 5/28/2012  Count = 6043
Date = 7/4/2012  Count = 7403
Date = 9/3/2012  Count = 6034
Date = 10/8/2012  Count = 5478
Date = 11/12/2012  Count = 6269
Date = 11/22/2012  Count = 2425
Date = 12/25/2012  Count = 1013


**e.** Write an aggregate query to calculate the average number of bikes rented for each season.

In [4]:
import pymongo

try:
    connection = pymongo.MongoClient('localhost', 27017)
    db = connection['test']

    cursor = db.bike.aggregate([
                {"$group": {"_id": "$season", "avgRental": {"$avg": "$rentalcount"}}}
             ])
    for record in cursor:
        print(record)
    
    connection.close()

except pymongo.errors.ConnectionFailure as e:
    print("Could not connect to MongoDB: %s" % e)

{'_id': 4, 'avgRental': 4728.162921348315}
{'_id': 3, 'avgRental': 5644.303191489362}
{'_id': 2, 'avgRental': 4992.33152173913}
{'_id': 1, 'avgRental': 2604.1325966850827}
